# Project 4: Predicting Fear on News headlines

## Problem statement : 

Using news headlines to train a classifer to predict whether Volatility Index(VIX) will go up or down on  daily basis  and a sentimental analysis tool to predict the extent of the movement of Volatility Index.

### Executive Summary

Efficient Market Hypothesis proposed by Fama [1965] states that stock market prices are driven by all observable information. In reality, it has been shown that investor sentiment can affect the asset prices due to the well-known psychological fact that investors with positive (negative) sentiment tend to make overly optimistic(pessimistic) judgments and decisions [Keynes, 1937]

Hence the purpose of this notebook is to check if the above hypthesis holds true and if the news sentiment can indeed affect the Volatilty price index and to what extent.

# Importing of Libraries

In [1]:
# get some libraries that will be useful
import re
import numpy as np # linear algebra
import pandas as pd
import seaborn as sns
import string
#To remove weekends from dataset
from pandas.tseries.offsets import BDay
# function to split the data for cross-validation
from sklearn.model_selection import train_test_split
#to filter out selected dates from dataset
import datetime

# Data Import

## Reddit Top 25 News Headlines

In [2]:
combined_news = pd.read_csv("../data/Combined_News_DJIA.csv")

In [3]:
combined_news.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [4]:
#We have 1989 rows and 27 columns
combined_news.shape  

(1989, 27)

# Data Cleaning and Preprocessing

In [5]:
#Extracting only columns top 25 news headlines and date from combined_news dataset and rename dataframe as X_df
features = [col for col in combined_news.columns if not col == 'Label']
X_df = combined_news[features]

In [6]:
#confirm that there is still 1989 rows, no weekends for this dataset!
X_df.shape 

(1989, 26)

In [7]:
#NEWS HEADLINES

In [8]:
# We will combine all 25 headlines into 1 column
X_df['all25'] = combined_news[combined_news.columns[1:]].apply (lambda x: ','.join(x.dropna().astype(str)),axis=1)
#convert them to lower case characters 
X_df['all25'] = X_df['all25'].str.lower()
#We only want the combined 25 headlines and our date column
X_df = X_df.filter(['Date','all25'],axis=1)
X_df.head()

,Date,all25
0,2008-08-08,"0,b""georgia 'downs two russian warplanes' as c..."
1,2008-08-11,"1,b'why wont america and nato help us? if they..."
2,2008-08-12,"0,b'remember that adorable 9-year-old who sang..."
3,2008-08-13,"0,b' u.s. refuses israel weapons to attack ira..."
4,2008-08-14,"1,b'all the experts admit that we should legal..."


# Importing Volatility Index Price

In [9]:
#Dataset consist of Y Variable(VIX PRICE) from 2004 to 2020
price = pd.read_csv("../data/vixcurrent.csv") 

Since our news headline datasets are from between June 8th, 2008 and July 1st, 2016, we would need to extract these date range out from Volatility Index Price.

In [10]:
#Index 1158 indicates price on 8th August 2008
price[price['Date'] == '8/8/2008'] 

,Date,VIX Open,VIX High,VIX Low,VIX Close
1158,8/8/2008,21.15,21.69,20.11,20.66


In [11]:
#Index 3146 indicates price on 1st July 2016
price [price['Date'] == '7/1/2016'] 

,Date,VIX Open,VIX High,VIX Low,VIX Close
3146,7/1/2016,15.59,15.86,14.61,14.77


In [12]:
#We know that the price range is between index 1158 and 3146
#Date is between 8 August 2008 and  1 July 2016
price = price.iloc[ 1158:3147 , : ]
price 
#We can see that there ar 1989 rows between these date ranges which matches to the number of rows for our news headlines dataset.

,Date,VIX Open,VIX High,VIX Low,VIX Close
1158,8/8/2008,21.15,21.69,20.11,20.66
1159,8/11/2008,20.66,20.96,19.66,20.12
1160,8/12/2008,20.64,21.51,20.38,21.17
1161,8/13/2008,21.57,22.11,20.80,21.55
1162,8/14/2008,22.30,22.30,20.07,20.34
...,...,...,...,...,...
3142,6/27/2016,24.38,26.72,22.93,23.85
3143,6/28/2016,21.76,22.07,18.75,18.75
3144,6/29/2016,18.12,18.27,16.48,16.64
3145,6/30/2016,16.91,16.99,15.29,15.63


In [13]:
#FEATURE Engineering to create our LABEL Column

In [14]:
#create a new column for the difference in the Closing and Opening Price
price['upordown'] = price['VIX Close'] - price['VIX Open']
#if closing price is higher then opening price, will assign value 1
price['upordown'] = np.where(price['upordown'] > 0,1, price['upordown'])
#if closing price is equals to opening price, will assign value 0
price['upordown'] = np.where(price['upordown'] == 0 ,0, price['upordown'])
#if closing price is lower than opening price, will assign value 0
price['upordown'] = np.where(price['upordown'] < 0,0, price['upordown'])

In [15]:
#updown column will be our Y variables for modelling with either 1 or 0.
price.head() 

,Date,VIX Open,VIX High,VIX Low,VIX Close,upordown
1158,8/8/2008,21.15,21.69,20.11,20.66,0.0
1159,8/11/2008,20.66,20.96,19.66,20.12,0.0
1160,8/12/2008,20.64,21.51,20.38,21.17,1.0
1161,8/13/2008,21.57,22.11,20.80,21.55,0.0
1162,8/14/2008,22.30,22.30,20.07,20.34,0.0


In [16]:
#We only want Date and upordown column from price in other to merge with our news dataset.
Y_feature = price.filter(['Date','upordown'], axis=1)
Y_feature.reset_index(drop=True, inplace=True)
Y_feature.head()

,Date,upordown
0,8/8/2008,0.0
1,8/11/2008,0.0
2,8/12/2008,1.0
3,8/13/2008,0.0
4,8/14/2008,0.0


In [17]:
#TIME TO MERGE OUR LABEL DATASET with ur news headlines.

In [18]:
#We merge 2 datafarme together with upordown as the price of VIX with the top 25 headings according to dates.
df = pd.merge(X_df, Y_feature, left_index=True, right_index=True, how='left')
#indicates columns have been successfully merged 
df

,Date_x,all25,Date_y,upordown
0,2008-08-08,"0,b""georgia 'downs two russian warplanes' as c...",8/8/2008,0.0
1,2008-08-11,"1,b'why wont america and nato help us? if they...",8/11/2008,0.0
2,2008-08-12,"0,b'remember that adorable 9-year-old who sang...",8/12/2008,1.0
3,2008-08-13,"0,b' u.s. refuses israel weapons to attack ira...",8/13/2008,0.0
4,2008-08-14,"1,b'all the experts admit that we should legal...",8/14/2008,0.0
...,...,...,...,...
1984,2016-06-27,"0,barclays and rbs shares suspended from tradi...",6/27/2016,0.0
1985,2016-06-28,"1,2,500 scientists to australia: if you want t...",6/28/2016,0.0
1986,2016-06-29,"1,explosion at airport in istanbul,yemeni form...",6/29/2016,0.0
1987,2016-06-30,"1,jamaica proposes marijuana dispensers for to...",6/30/2016,0.0


In [19]:
#We drop 'Date_y' column as it is not required. 
df.drop(columns=['Date_y'],inplace = True)
#We then rename the column Date_x into Date.
df.rename(columns={"Date_x": "Date"},inplace= True)

In [20]:
#we will drop off the rest of the columns other than Date
df

,Date,all25,upordown
0,2008-08-08,"0,b""georgia 'downs two russian warplanes' as c...",0.0
1,2008-08-11,"1,b'why wont america and nato help us? if they...",0.0
2,2008-08-12,"0,b'remember that adorable 9-year-old who sang...",1.0
3,2008-08-13,"0,b' u.s. refuses israel weapons to attack ira...",0.0
4,2008-08-14,"1,b'all the experts admit that we should legal...",0.0
...,...,...,...
1984,2016-06-27,"0,barclays and rbs shares suspended from tradi...",0.0
1985,2016-06-28,"1,2,500 scientists to australia: if you want t...",0.0
1986,2016-06-29,"1,explosion at airport in istanbul,yemeni form...",0.0
1987,2016-06-30,"1,jamaica proposes marijuana dispensers for to...",0.0


In [21]:
# our final dataset has been done! 

## Saving our dataframe for modelling, before that lets proceed to part 2 for modelling.

In [23]:
df.to_csv('../data/final_dataframe.csv', index=False)